### First little Network
This class explores a first little neural network structure for the CT data.

In [ ]:
%matplotlib notebook
import tensorflow as tf
import numpy as np
import dicom
import os
from matplotlib import pyplot as plt, cm
import time
from matplotlib import animation, rc
from cvloop import cvloop
import itertools

In [ ]:
def read_patient(path):
    """
    Returns a sorted List of scans that are found in one specific folder.
    """
    all_scans = []  # create an empty list
    for dirName, subdirList, fileList in os.walk(path):
        for filename in fileList:
            if ".dcm" in filename.lower():  # check whether the file's DICOM
                all_scans.append(dicom.read_file(os.path.join(dirName,filename)))
    all_scans.sort(key = lambda x: int(x.InstanceNumber))
    # TODO: normalize images!!
    return all_scans

def conv2array(scan_files):
    """
    Converts the dicom files of one patient to a numpy array. 
    """
    ref_scan = scan_files[0]
    # Load dimensions based on the number of rows, columns, and slices (along the Z axis)
    pixel_dims = (int(ref_scan.Rows), int(ref_scan.Columns), len(scan_files))

    # Load spacing values (in mm)
    pixel_spacing = (float(ref_scan.PixelSpacing[0]), float(ref_scan.PixelSpacing[1]), float(ref_scan.SliceThickness))

    x = np.arange(0.0, (pixel_dims[0]+1)*pixel_spacing[0], pixel_spacing[0])
    y = np.arange(0.0, (pixel_dims[1]+1)*pixel_spacing[1], pixel_spacing[1])
    z = np.arange(0.0, (pixel_dims[2]+1)*pixel_spacing[2], pixel_spacing[2])

    # The array is sized based on 'pixel_dims'
    array_imgs = np.zeros(pixel_dims, dtype=ref_scan.pixel_array.dtype)

    # loop through all the DICOM files
    for i in range(len(scan_files)):
        # store the raw image data
        array_imgs[:, :, i] = scan_files[i].pixel_array
    return array_imgs

In [ ]:
scan_pat = read_patient("../../data/LIDC-IDRI-0666/1.3.6.1.4.1.14519.5.2.1.6279.6001.150264634200093580367988090366/1.3.6.1.4.1.14519.5.2.1.6279.6001.325580698241281352835338693869/")
array_patient = conv2array(scan_pat)

In [ ]:
class data:
    def __init__(self, scans):
        self.scans = scans
        self.i = 0
            
    def read(self):
        self.i = self.i + 1
        return True, self.scans[:,:,self.i]
    
cvloop(data(array_patient), function=lambda x: 1-x, side_by_side=True, print_info=True)

In [ ]:
# TensorFlow Graph Definition

def get_input(patient_scan, qube_size=(5,5,3)):
    """
    Get a random cube out of a random scan (?) and have the network put out 0 or 1 for nodule or not?
    But that would be underrepresenting the nodule containing segments.. So first compute a database of nodules
    and a database of non-nodule patches? And in the end have a algorithm that goes over the 
    whole scan and outputs 1, 0 for the whole thing?! 
    """
    return patient_scan
    
    

def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
    shape = [filter_size, filter_size,  num_input_channels, num_filters]
 
    weights = new_weights(shape=shape)
 
    biases = new_biases(length=num_filters)
 
    layer = tf.nn.conv3d(input=input, 
                         filter=weights, 
                         strides=[1, 1, 1, 1], 
                         padding='SAME')
    layer += biases
 
    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
    layer = tf.nn.relu(layer)
    return layer, weights


def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
 
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features


def new_fc_layer(input,      
             num_inputs,     
             num_outputs,    
             use_relu=True): 

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:
img_size_flat= img_size * img_size * num_channels
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
 
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.shape(a))

In [ ]:
array_patient.shape

In [ ]:
# think about (train 9/10, validation 1/10) 2/3 and test 1/3
# train until performance get's worse on validation set